In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import hamming_loss, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import os
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict
from sklearn.metrics import classification_report, hamming_loss
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from sklearn.multioutput import MultiOutputClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#!pip install catboost
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [2]:
def merge_csv_files(folder_path):
    all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dataframes = [pd.read_csv(file) for file in all_files]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [5]:
labelled_folder = r"C:\github\ADL\adl_german_news\crawler and data\src\preprocess data\data_ready_for_analysis\labelled"

In [6]:
labelled_data = merge_csv_files(labelled_folder)

In [7]:
labelled_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,"[1, 1, 1, 0]"
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,"[1, 1, 0, 0]"
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,"[0, 1, 0, 0]"
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,"[0, 0, 0, 1]"
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,"[0, 0, 0, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,"[0, 0, 0, 1]"
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,"[0, 0, 0, 1]"
338,welt,https://www.welt.de/politik/ausland/article242...,"krieg, sp, ukraine, ns, spd, rki, schi, ss, rn...",2022-12-10,2022-12-10,2024-10-20,NaN,NaN,"Newsteam, Leopard 2, Russland-Ukraine-Krieg (2...",Leopard 2: USA fuer Lieferung deutscher Kampfp...,Kanzler Scholz will moderne westliche Kampfpan...,Kanzler Scholz will moderne westliche Kampfpan...,0.0,0.0,0.0,1.0,2616.0,True,"[0, 0, 0, 1]"
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,"[0, 0, 0, 1]"


In [8]:
validation_data = labelled_data.sample(n=30, random_state=42)
remaining_data = labelled_data.drop(validation_data.index)

In [ ]:
X_val = validation_data[["title", "description", "body"]].apply(
    lambda row: " ".join(row.values.astype(str)), axis=1
)
y_val = validation_data[["Fake News", "Extreme bias", "clickbait", "credible"]]

In [9]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
validation_data[labels].sum()

Fake News        7.0
Extreme bias     7.0
clickbait        5.0
credible        23.0
dtype: float64

In [10]:
labels = ['Fake News', 'Extreme bias', 'clickbait', 'credible']
remaining_data[labels].sum()

Fake News        75.0
Extreme bias     98.0
clickbait        96.0
credible        206.0
dtype: float64

In [13]:
unlabelled_folder = r"C:\github\ADL\adl_german_news\crawler and data\src\preprocess data\data_ready_for_analysis\unlabelled"

In [14]:
unlabelled_data = merge_csv_files(unlabelled_folder)

In [15]:
label_mapping = {
    "Fake News": 1,
    "Extreme bias": 2,
    "clickbait": 3,
    "credible": 0
}

def map_labels(row):
    labels = [label_mapping[label] for label in label_mapping if row[label] == 1]
    return "-".join(map(str, sorted(labels))) if labels else "none"

remaining_data["label"] = remaining_data.apply(map_labels, axis=1)
remaining_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,1-2-3
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,1-2
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,2
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,0
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,welt,https://www.welt.de/iconist/mode/article249414...,"sp, ns, spo, schi, demo, ss, rna",2024-01-10,2024-01-10,2024-10-20,NaN,NaN,"texttospeech, Uniqlo Wachsjacke, Timberland, M...",Mode: Diese modischen Investitionen lohnen sic...,Die Temperaturen? Unterirdisch eisig! Doch in ...,Die Temperaturen? Unterirdisch eisig! Doch in ...,0.0,0.0,0.0,1.0,4587.0,True,0
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,0
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,0
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,0


In [ ]:
le = LabelEncoder()
remaining_data['label']= le.fit_transform(remaining_data['label']) 
remaining_data

,news_outlet,provenance,query_keywords,creation_date,last_modified,crawl_date,author_person,author_organization,news_keywords,title,description,body,Fake News,Extreme bias,clickbait,credible,body_len,has_label,label
0,bild,https://www.bild.de/politik/ausland/politik-au...,"linke, migration, csu, cdu",2022-06-15,2022-06-15,2024-10-20,Filipp Piatov und Ralf Schuler,NaN,NaN,Sie verspottete Deutsche als Kartoffeln: Aktiv...,Schon wieder sorgt eine Personalentscheidung d...,Schon wieder sorgt eine Personalentscheidung d...,1.0,1.0,1.0,0.0,3574.0,True,5
1,bild,https://www.bild.de/politik/ausland/politik-au...,israel,2023-11-10,2023-11-10,2024-10-20,"P. Tiede, A. Link und H.-J. Vehlewald",NaN,NaN,Reporter als Terror-Helfer: Die perfide Propag...,"Es ist der 7. Oktober 2023. Der Tag, der alles...","Es ist der 7. Oktober 2023. Der Tag, der alles...",1.0,1.0,0.0,0.0,5226.0,True,4
2,bild,https://www.bild.de/politik/inland/politik-inl...,"fdp, csu, spd, cdu, afd, annalena baerbock, ch...",2023-06-19,2023-06-19,2024-10-20,Filipp Piatov,NaN,"CDU, CSU, die Gruenen, SPD, Wuest Hendrik, Mer...",Streit in der CDU: Umfrage-Schlacht zwischen M...,In der CDU liegen die Nerven blank! Es tobt ei...,In der CDU liegen die Nerven blank! Spaetesten...,0.0,1.0,0.0,0.0,3071.0,True,7
3,bild,https://www.bild.de/politik/ausland/politik-au...,"israel, spd, olaf scholz",2023-10-30,2023-10-30,2024-10-20,NaN,BILD,"Gaza-Streifen, Gaza, Israel, Hamas, Nahost-Kon...",Shani Louk ermordet Scholz: Zeigt die ganze B...,Shani Louk wurde barbarisch ermordet. Das best...,Es ist das pure Grauen. Am 7. Oktober verschle...,0.0,0.0,0.0,1.0,2655.0,True,0
4,bild,https://www.bild.de/regional/leipzig/leipzig-n...,"abschiebung, migration, cdu",2023-11-09,2023-11-09,2024-10-20,KARL KEIM,NaN,"Tunesien, Georgien, Sachsen, asylrecht, Schust...",Sachsen: Nur jede dritte Abschiebung findet wi...,In Sachsen klappt nur jede dritte Abschiebung....,Leipzig In Sachsen wurden bisher in diesem Ja...,0.0,0.0,0.0,1.0,2447.0,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,welt,https://www.welt.de/iconist/mode/article249414...,"sp, ns, spo, schi, demo, ss, rna",2024-01-10,2024-01-10,2024-10-20,NaN,NaN,"texttospeech, Uniqlo Wachsjacke, Timberland, M...",Mode: Diese modischen Investitionen lohnen sic...,Die Temperaturen? Unterirdisch eisig! Doch in ...,Die Temperaturen? Unterirdisch eisig! Doch in ...,0.0,0.0,0.0,1.0,4587.0,True,0
336,welt,https://www.welt.de/politik/ausland/article242...,"krieg, christ, ss, sp, ukraine, ns, spo, nazi,...",2022-12-12,2022-12-12,2024-10-20,NaN,NaN,"Newsteam, Russland-Ukraine-Krieg (24.2.2022), ...",Ukraine-News ++ Ukraines Verteidigungsminister...,Laut dem ukrainischen Verteidigungsminister bi...,Laut dem ukrainischen Verteidigungsminister bi...,0.0,0.0,0.0,1.0,21564.0,True,0
337,welt,https://www.welt.de/politik/deutschland/articl...,"sp, terror, ns, spd, schi, rechtsextremismus, ss",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Newsteam, Wehrbeauftragte, Hoegl, Eva, Reichsb...",Wehrbeauftragte fordert haerteres Vorgehen geg...,Nach der Razzia gegen ein mutmassliches Terror...,Nach der Razzia gegen ein mutmassliches Terror...,0.0,0.0,0.0,1.0,972.0,True,0
339,welt,https://www.welt.de/politik/deutschland/articl...,"terror, ss, ns",2022-12-13,2022-12-13,2024-10-20,NaN,NaN,"Schindler-Frederik, Bundestag WELT, AfD, Razzi...",Reichsbuerger-Razzia: Ermittler finden 93 Waff...,Bei der bundesweiten Razzia gegen eine Reichsb...,Bei der bundesweiten Razzia gegen eine Reichsb...,0.0,0.0,0.0,1.0,1464.0,True,0


In [ ]:
X = remaining_data[['title', 'description', 'body']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
y = remaining_data[['label']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [18]:
unique_labels, counts = np.unique(y, return_counts=True)

for label, count in zip(unique_labels, counts):
    print(f"Label: {label}, Count: {count}")

Label: 0, Count: 183
Label: 1, Count: 6
Label: 2, Count: 17
Label: 3, Count: 3
Label: 4, Count: 20
Label: 5, Count: 50
Label: 6, Count: 2
Label: 7, Count: 3
Label: 8, Count: 19
Label: 9, Count: 8


In [19]:
print("Unique classes after encoding:", np.unique(y_train))

Unique classes after encoding: [0 1 2 3 4 5 6 7 8 9]


In [20]:
print("Unique classes after encoding:", np.unique(y_test))

Unique classes after encoding: [0 1 2 3 4 5 7 8 9]


In [21]:
vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [23]:

results = {}

gb_model = MultiOutputClassifier(GradientBoostingClassifier(random_state=42))
gb_model.fit(X_train_tfidf, y_train)
gb_pred = gb_model.predict(X_test_tfidf)
gb_acc = accuracy_score(y_test, gb_pred)
results['MultiOutputClassifier'] = gb_acc
print(f"Accuracy with MultiOutputClassifier: {gb_acc}")

xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train_tfidf, y_train)
xgb_pred = xgb_model.predict(X_test_tfidf)
xgb_acc = accuracy_score(y_test, xgb_pred)
results['XGBoost'] = xgb_acc
print(f"Accuracy with XGBoost: {xgb_acc}")

lgbm_model = LGBMClassifier(random_state=42)
lgbm_model.fit(X_train_tfidf, y_train)
lgbm_pred = lgbm_model.predict(X_test_tfidf)
lgbm_acc = accuracy_score(y_test, lgbm_pred)
results['LightGBM'] = lgbm_acc
print(f"Accuracy with LightGBM: {lgbm_acc}")

catboost_model = CatBoostClassifier(random_state=42, verbose=0)
catboost_model.fit(X_train_tfidf, y_train)
catboost_pred = catboost_model.predict(X_test_tfidf)
catboost_acc = accuracy_score(y_test, catboost_pred)
results['CatBoost'] = catboost_acc
print(f"Accuracy with CatBoost: {catboost_acc}")

bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100)
bagging_model.fit(X_train_tfidf, y_train)
bagging_pred = bagging_model.predict(X_test_tfidf)
bagging_acc = accuracy_score(y_test, bagging_pred)
results['BaggingClassifier_DecisionTree'] = bagging_acc
print(f"Accuracy with BaggingClassifier (DecisionTree): {bagging_acc}")

print("\nSummary of Model Accuracies:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 4.2 MB/s eta 0:00:24
   ---------------------------------------- 0.5/101.7 MB 4.2 MB/s eta 0:00:24
   ---------------------------------------- 0.5/101.7 MB 4.2 MB/s eta 0:00:24
   ---------------------------------------- 0.5/101.7 MB 4.2 MB/s eta 0:00:24
   ---------------------------------------- 0.8/101.7 MB 568.6 kB/s eta 0:02:58
   ---------------------------------------- 1.0/101.7 MB 740.5 kB/s eta 0:02:16
   ---------------------------------------- 1.0/101.7 MB 740.5 kB/s eta 0:02:16
   ---------------------------------------- 1.0/101.7 MB 740.5 kB/s eta 0:02:16
    --------------------------------------- 1.3/101.7 MB 627.1 kB/s eta 0:02:41
    --------------------------------------- 1.6/101.7 MB 687.7 kB/s eta 0:02:26
    --------------------------------------- 1.8/101.7 MB 719.2 kB/s eta 0:02:19
   - -------------------------------------- 2.6/101.7 MB 

c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002779 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 9511
[LightGBM] [Info] Number of data points in the train set: 248, number of used features: 495
[LightGBM] [Info] Start training from score -0.529822
[LightGBM] [Info] Start training from score -3.903991
[LightGBM] [Info] Start training from score -2.874371
[LightGBM] [Info] Start training from score -4.820282
[LightGBM] [Info] Start training from score -2.740840
[LightGBM] [Info] Start training from score -1.824549
[LightGBM] [Info] Start training from score -4.820282
[LightGBM] [Info] Start training from score -4.820282
[LightGBM] [Info] Start training from score -2.805379
[LightGBM] [Info] Start training from score -3.721669
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy with BaggingClassifier (DecisionTree): 0.5714285714285714

Summary of Model Accuracies:
MultiOutputClassifier: 0.5714
XGBoost: 0.5556
LightGBM: 0.5714
CatBoost: 0.5873
BaggingClassifier_DecisionTree: 0.5714


In [24]:
results = {}

bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100)
bagging_model.fit(X_train_tfidf, y_train)
bagging_pred = bagging_model.predict(X_test_tfidf)
bagging_acc = accuracy_score(y_test, bagging_pred)
results['BaggingClassifier_DecisionTree'] = bagging_acc
print(f"Accuracy with BaggingClassifier (DecisionTree): {bagging_acc}")

from sklearn.neighbors import KNeighborsClassifier
bagging_knn = BaggingClassifier(estimator=KNeighborsClassifier(), random_state=42, n_estimators=100)
bagging_knn.fit(X_train_tfidf, y_train)
bagging_knn_pred = bagging_knn.predict(X_test_tfidf)
bagging_knn_acc = accuracy_score(y_test, bagging_knn_pred)
results['BaggingClassifier_KNeighbors'] = bagging_knn_acc
print(f"Accuracy with BaggingClassifier (KNeighbors): {bagging_knn_acc}")

from sklearn.linear_model import LogisticRegression
bagging_lr = BaggingClassifier(estimator=LogisticRegression(), random_state=42, n_estimators=100)
bagging_lr.fit(X_train_tfidf, y_train)
bagging_lr_pred = bagging_lr.predict(X_test_tfidf)
bagging_lr_acc = accuracy_score(y_test, bagging_lr_pred)
results['BaggingClassifier_LogisticRegression'] = bagging_lr_acc
print(f"Accuracy with BaggingClassifier (LogisticRegression): {bagging_lr_acc}")

print("\nSummary of Model Accuracies:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy with BaggingClassifier (DecisionTree): 0.5714285714285714


c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy with BaggingClassifier (KNeighbors): 0.47619047619047616


c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_bagging.py:888: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy with BaggingClassifier (LogisticRegression): 0.5873015873015873

Summary of Model Accuracies:
BaggingClassifier_DecisionTree: 0.5714
BaggingClassifier_KNeighbors: 0.4762
BaggingClassifier_LogisticRegression: 0.5873


In [25]:
results = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)


models = {
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(random_state=42),
    "CatBoost": CatBoostClassifier(random_state=42, verbose=0),
    "BaggingClassifier_DecisionTree": BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42, n_estimators=100),
    "BaggingClassifier_KNeighbors": BaggingClassifier(estimator=KNeighborsClassifier(), random_state=42, n_estimators=100),
    "BaggingClassifier_LogisticRegression": BaggingClassifier(estimator=LogisticRegression(), random_state=42, n_estimators=100)
}

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    scores = cross_val_score(model, X_train_tfidf, y_train, cv=kf, scoring='accuracy')
    results[model_name] = {
        "mean_accuracy": np.mean(scores),
        "cv_scores": scores
    }
    
    y_pred_cv = cross_val_predict(model, X_train_tfidf, y_train, cv=kf)
    print(f"Cross-Validated Accuracy for {model_name}: {np.mean(scores):.4f}")
    print("Classification Report:")
    print(classification_report(y_train, y_pred_cv))

print("\nSummary of Model Accuracies:")
for model_name, result in results.items():
    print(f"{model_name}: Mean Accuracy: {result['mean_accuracy']:.4f}")

Evaluating GradientBoostingClassifier...


c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.p

Cross-Validated Accuracy for GradientBoostingClassifier: 0.5198
Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.84      0.70       146
           1       1.00      0.20      0.33         5
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        16
           5       0.12      0.05      0.07        40
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         2
           8       0.43      0.20      0.27        15
           9       0.00      0.00      0.00         6

    accuracy                           0.52       248
   macro avg       0.21      0.13      0.14       248
weighted avg       0.42      0.52      0.45       248

Evaluating XGBoost...


c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\hanna\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6 7 8], got [0 1 2 4 5 6 7 8 9]

There are issues concerning the different labels. As this approach is also worse (what is looks like now), I will leave it like this and not continue